In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet121(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 1, 1, 1024)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(3931, 1024)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.9s


[CV] ........................... C=0.01, score=0.974555, total=   2.3s
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.947837, total=   2.3s
[CV] ........................... C=0.01, score=0.944020, total=   2.3s
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.941476, total=   7.0s
[CV] C=1000.0 ........................................................
[CV] ........................... C=10.0, score=0.930025, total=   6.7s
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.963104, total=   6.7s
[CV] C=10000.0 .......................................................
[CV] ............................ C=1.0, score=0.932570, total=   7.1s
[CV] C=10000.0 .......................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    8.3s


[CV] ........................... C=10.0, score=0.922392, total=   7.6s
[CV] C=100000.0 ......................................................
[CV] ............................ C=0.1, score=0.944091, total=   8.4s
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.956743, total=   7.5s
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.930025, total=   7.7s
[CV] C=1000000.0 .....................................................
[CV] ............................ C=0.1, score=0.941476, total=   8.5s
[CV] ............................ C=1.0, score=0.926209, total=   8.3s
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] ......................... C=1000.0, score=0.930025, total=   6.5s
[CV] C=10000000.0 ....................................................
[CV] .

...

[CV] .................. C=10000000000.0, score=0.924936, total=   4.0s
[CV] .................... C=100000000.0, score=0.924936, total=   6.5s
[CV] ................... C=1000000000.0, score=0.923761, total=   6.2s
[CV] .................... C=100000000.0, score=0.924936, total=   6.8s
[CV] ................... C=1000000000.0, score=0.935115, total=   6.3s
[CV] ................... C=1000000000.0, score=0.924936, total=   6.1s
[CV] ................... C=1000000000.0, score=0.963104, total=   6.2s
[CV] ................... C=1000000000.0, score=0.924936, total=   6.2s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   21.2s remaining:    1.8s


[CV] .................. C=10000000000.0, score=0.932570, total=   5.4s
[CV] .................. C=10000000000.0, score=0.926302, total=   5.8s
[CV] .................. C=10000000000.0, score=0.956743, total=   5.6s
[CV] .................. C=10000000000.0, score=0.924936, total=   5.3s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   21.9s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([2.02469354, 8.59994922, 7.53208761, 7.45145907, 7.19395123,
        7.43699002, 6.78945718, 8.08289371, 7.58439422, 6.9259726 ,
        7.11361618, 6.1637599 , 5.21577601]),
 'mean_score_time': array([0.05251884, 0.07270083, 0.06709633, 0.04003949, 0.06354055,
        0.07291417, 0.06325269, 0.07840571, 0.03386812, 0.05525904,
        0.03268576, 0.0307847 , 0.00622711]),
 'mean_test_score': array([0.95242941, 0.94657848, 0.93691173, 0.9330959 , 0.93207835,
        0.9330959 , 0.93284152, 0.93284152, 0.93284152, 0.93436785,
        0.93335029, 0.93436785, 0.9330959 ]),
 'mean_train_score': array([0.9793309 , 0.99287716, 0.99955483, 0.99968201, 0.99974563,
        0.99923684, 0.99930044, 0.99930044, 0.99930044, 0.99961842,
        0.99949122, 0.99968201, 0.99974563]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    10000000000.0],
    

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.95243, std: 0.01119, params: {'C': 0.01},
 mean: 0.94658, std: 0.01100, params: {'C': 0.1},
 mean: 0.93691, std: 0.01298, params: {'C': 1.0},
 mean: 0.93310, std: 0.01523, params: {'C': 10.0},
 mean: 0.93208, std: 0.01278, params: {'C': 100.0},
 mean: 0.93310, std: 0.01333, params: {'C': 1000.0},
 mean: 0.93284, std: 0.01339, params: {'C': 10000.0},
 mean: 0.93284, std: 0.01339, params: {'C': 100000.0},
 mean: 0.93284, std: 0.01339, params: {'C': 1000000.0},
 mean: 0.93437, std: 0.01494, params: {'C': 10000000.0},
 mean: 0.93335, std: 0.01503, params: {'C': 100000000.0},
 mean: 0.93437, std: 0.01494, params: {'C': 1000000000.0},
 mean: 0.93310, std: 0.01216, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9524
